## Library Imports

In [2]:
library(glue)
library(DAAG)

options(repr.plot.width=10, repr.plot.height=5)

set.seed(20)
options(warn=-1)

Loading required package: lattice


### Question 7.1

**Describe a situation or problem from your job, everyday life, current events, etc., for which exponential
smoothing would be appropriate. What data would you need? Would you expect the value of α (the
first smoothing parameter) to be closer to 0 or 1, and why?**

**ANSWER:**
The first that comes to mind when thinking of exponential smoothing and its uses for short-term forecasting is the stock market. Nowadays, there are numerous firms that try to use machine learning/AI to predict the prices of stocks in order to make lots of profit.

The data I would need is stock price data in minute or maybe even second intervals. Seeing as how volatile stock prices are, I would say the alpha value would be near 1 so a lot of weight would be placed on the true current observation and little weight on the previous observation.

### Question 7.2
**Using the 20 years of daily high temperature data for Atlanta (July through October) from Question 6.2
(file temps.txt), build and use an exponential smoothing model to help make a judgment of whether
the unofficial end of summer has gotten later over the 20 years. (Part of the point of this assignment is
for you to think about how you might use exponential smoothing to answer this question. Feel free to
combine it with other models if you’d like to. There’s certainly more than one reasonable approach.)
Note: in R, you can use either HoltWinters (simpler to use) or the smooth package’s es function
(harder to use, but more general). If you use es, the Holt-Winters model uses model=”AAM” in the
function call (the first and second constants are used “A”dditively, and the third (seasonality) is used
“M”ultiplicatively; the documentation doesn’t make that clear).**

In [3]:
temp_data <- read.table('../data/7.2tempsSummer2018.txt', sep='', header=TRUE)

temp_data$datetime <- as.Date(temp_data$DAY, format = "%d-%b")

First, I transformed the data into a time series data frame with a frequency of 123 since there are 123 days in the original data set.

In [44]:
temp_ts  <- ts(as.vector(temp_data[,c(2:21)]), start=1996, frequency=123)

The transformed data set was then inputted into the Holt Winters function and the smoothed data was plotted as a function of time.

**The plot is in the appendix.**

In [126]:
hw <- HoltWinters(temp_ts)
plot.ts(fitted(hw)[,1], col=3, ylab="Temperature (F)", xlab="Year")

The resulting seasonal factors were transformed into a matrix and used in the CUSUM approach to find the date at which summer ended.

In [38]:
m <- matrix(hw$fitted[,4], ncol=123)

Custom CUSUM function that I coded in R is below.

In [109]:
# cusum_approach takes in the data of interest, threshold (T), critical value (C), mean (mu), 
# and a boolean value for is_inc.
# The is_inc determines whether the change that is being detected is an increase or decrease.
cusum_approach <- function(data, T, C, mu, is_inc) {
    S_t <- 0
    end_index <- length(data)
    
    if (isTRUE(is_inc)) {
        for (i in 1:length(data)) {
            S_t <- S_t - mu + data[i] - C
            # Check if the S_t is less than 0, if it is then it should be 
            # reset to 0
            if (S_t <= 0) {
                S_t <- 0
                }
            # Check if S_t is greater than the threshold, if it is then 
            # the loop ends
            if (S_t > T) {
                end_index <- i
                break
            }
        }
    }

    else {
        for (i in 1:length(data)) {
            S_t <- S_t + mu - data[i] - C
            # Check if the S_t is less than 0, if it is then it should be 
            # reset to 0
            if (S_t <= 0) {
                S_t <- 0
                }
            # Check if S_t is greater than the threshold, if it is then 
            # the loop ends
            if (S_t > T) {
                end_index <- i
                break
            }
        }
    }
    return(end_index)
}

I tested a variety of C and T values and this combination of C = 1 and T = 25 seemed to give the most reasonable summer end dates.

In [119]:
C <- 1
T <- 20

for (i in 1:nrow(m)) {
    year_data <- m[i,]

    mu <- mean(year_data)
    end_index <- cusum_approach(year_data, T, C, mu, is_inc=FALSE)
    
    summer_end_date <- temp_data[end_index,1]
    year <- substr(colnames(temp_ts)[i], 2, 5)
    print(glue("Summer ends on {summer_end_date} in the year {year}"))    
}

Summer ends on 7-Jul in the year 1996
Summer ends on 7-Jul in the year 1997
Summer ends on 7-Jul in the year 1998
Summer ends on 6-Jul in the year 1999
Summer ends on 26-Jul in the year 2000
Summer ends on 13-Jul in the year 2001
Summer ends on 6-Jul in the year 2002
Summer ends on 3-Sep in the year 2003
Summer ends on 26-Jul in the year 2004
Summer ends on 8-Aug in the year 2005
Summer ends on 13-Jul in the year 2006
Summer ends on 13-Jul in the year 2007
Summer ends on 12-Jul in the year 2008
Summer ends on 1-Aug in the year 2009
Summer ends on 6-Jul in the year 2010
Summer ends on 6-Jul in the year 2011
Summer ends on 6-Jul in the year 2012
Summer ends on 27-Aug in the year 2013
Summer ends on 14-Aug in the year 2014


For the same C and T values, I ran the original data through the CUSUM approach to compare the summer end dates before and after exponential smoothing.

In [121]:
for (i in 2:21) {
    # Initialize the mean (mu)
    mu <- mean(temp_data[,i])

    # Get the index at which the threshold is passed.
    end_index <- cusum_approach(temp_data[,i], T, C, mu, is_inc=FALSE)

    summer_end_date <- temp_data[end_index,1]
    year <- substr(colnames(temp_data)[i], 2, 5)
    print(glue("Summer ends on {summer_end_date} in the year {year}"))
    
    }

Summer ends on 21-Sep in the year 1996
Summer ends on 25-Sep in the year 1997
Summer ends on 30-Sep in the year 1998
Summer ends on 20-Sep in the year 1999
Summer ends on 6-Sep in the year 2000
Summer ends on 25-Sep in the year 2001
Summer ends on 25-Sep in the year 2002
Summer ends on 29-Sep in the year 2003
Summer ends on 19-Sep in the year 2004
Summer ends on 7-Oct in the year 2005
Summer ends on 21-Sep in the year 2006
Summer ends on 17-Sep in the year 2007
Summer ends on 21-Sep in the year 2008
Summer ends on 2-Oct in the year 2009
Summer ends on 28-Sep in the year 2010
Summer ends on 7-Sep in the year 2011
Summer ends on 30-Sep in the year 2012
Summer ends on 16-Aug in the year 2013
Summer ends on 26-Sep in the year 2014
Summer ends on 16-Sep in the year 2015


The exponential smoothing provided summer end dates that are much earlier than the end dates estimated on the original data set. This was actually counter intuitive for me. I expected the end dates to actually be later after exponential smoothing because the threshold would be harder to surpass if the data was more stable. However, the opposite was true. My hypothesis as to why this happened is that the volatility in the temperature data causes the S_t value to fluctuate a lot. So, the S_t value can come close to the threshold value but if the next day a sudden drop in temperature happened then S_t could now be very far from the threshold. The exponential smoothing prevents such sudden changes in S_t by smoothing out the temperature data.

### Question 8.1
**Describe a situation or problem from your job, everyday life, current events, etc., for which a linear
regression model would be appropriate. List some (up to 5) predictors that you might use.**

**ANSWER:**

Linear regression is useful when predicting house prices. Especially in the Bay Area where the housing market is so high, its important to be able to accurately predict the housing price so that you don't overpay on an already ridiculously highly priced house. 

5 predictors to use:
1. Square footage
2. Distance of bus stop or subway stations
3. Year it was built
4. Number of bedrooms
5. Number of bathrooms

### Question 8.2
**Using crime data from http://www.statsci.org/data/general/uscrime.txt (file uscrime.txt,
description at http://www.statsci.org/data/general/uscrime.html ), use regression (a useful R function is
lm or glm) to predict the observed crime rate in a city with the following data:**

- M = 14.0
- So = 0
- Ed = 10.0
- Po1 = 12.0
- Po2 = 15.5
- LF = 0.640
- M.F = 94.0
- Pop = 150
- NW = 1.1
- U1 = 0.120
- U2 = 3.6
- Wealth = 3200
- Ineq = 20.1
- Prob = 0.04
- Time = 39.0

**Show your model (factors used and their coefficients), the software output, and the quality of fit.
Note that because there are only 47 data points and 15 predictors, you’ll probably notice some
overfitting. We’ll see ways of dealing with this sort of problem later in the course.**

In [8]:
crime_data <- read.table('../data/5.1uscrimeSummer2018.txt', sep='', header=TRUE)

In [9]:
M  <-  14.0
So <- 0
Ed <- 10.0
Po1 <- 12.0
Po2 <- 15.5
LF <- 0.640
M.F <- 94.0
Pop <- 150
NW <- 1.1
U1 <- 0.120
U2 <- 3.6
Wealth <- 3200
Ineq <- 20.1
Prob <- 0.04
Time <- 39.0

sample_data <- data.frame(M,So,Ed,Po1,Po2,LF,M.F,Pop,NW,U1,U2,Wealth,Ineq,Prob,Time)

Use cross-validation to determine the root mean squared error (RMSE) and the mean absolute error (MAE) for each fold in the cross-validation step.

In [10]:
# RMSE function
rmse <- function(resids) {
    sqrt(mean(resids^2))
}
# MAE function
mae <- function(resids) {
    mean(abs(resids))
}

In [11]:
# Shuffle data
shuffled_df <- crime_data[sample(nrow(crime_data)), ]

# 5 fold split
folds <- cut(seq(1, nrow(crime_data)), breaks=5, labels=FALSE)

# For each fold, train and test on Linear Regression algorithm
resids <- c(1:nrow(crime_data))
preds <- c(1:nrow(crime_data))
for (i in 1:5) {
    print(glue("Iteration #{i}"))
    test_indices <- which(folds==i, arr.ind=TRUE)
    test_data <- crime_data[test_indices, ]
    train_data <- crime_data[-test_indices, ]

    temp_fit <- lm(formula=Crime ~., data=train_data)
    print(summary(temp_fit))
    
    # Perform prediction step on test data since model was fit using the training data
    preds[test_indices] <- predict(temp_fit, test_data)    
    resids[test_indices] <- test_data$Crime - preds[test_indices]
}

overall_rmse <- round(rmse(resids), 3)
overall_mae <- round(mae(resids), 3)

print(glue("RMSE: {overall_rmse}"))
print(glue("MAE: {overall_mae}"))

Iteration #1

Call:
lm(formula = Crime ~ ., data = train_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-354.88  -86.44   -4.82   77.37  467.47 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)   
(Intercept) -6104.9296  2068.7312  -2.951  0.00763 **
M             102.2760    60.1488   1.700  0.10382   
So           -114.2915   195.7346  -0.584  0.56550   
Ed            120.1692    77.2415   1.556  0.13471   
Po1           174.3027   118.6461   1.469  0.15663   
Po2          -112.9272   131.6819  -0.858  0.40081   
LF           -764.5198  1700.0301  -0.450  0.65753   
M.F            15.9682    24.5602   0.650  0.52263   
Pop            -1.2496     1.6571  -0.754  0.45919   
NW              4.3656     7.9608   0.548  0.58921   
U1          -5307.2621  5371.9166  -0.988  0.33442   
U2            148.8000   103.3013   1.440  0.16448   
Wealth          0.2596     0.1493   1.739  0.09666 . 
Ineq           77.4564    28.8980   2.680  0.01401 * 
Prob        -3938.

I also want to see the residual plot as well for the predicted values.

**The plot is in the appendix.**

In [128]:
xyplot(resids ~ preds,
  xlab = "Fitted Values",
  ylab = "Residuals",
  main = "Residual Diagnostic Plot",
  panel = function(x, y, ...)
  {
    panel.grid(h = -1, v = -1)
    panel.abline(h = 0)
    panel.xyplot(x, y, ...)
  }
)

From the summary of the models above, it seems that the features that seemed most useful were Ineq, Ed, M, Po1, U2, Prob, and Time. Using this subset of predictors, I'll build another linear regression model.

In [13]:
# Shuffle data
shuffled_df <- crime_data[sample(nrow(crime_data)), ]

# 5 fold split
folds <- cut(seq(1, nrow(crime_data)), breaks=5, labels=FALSE)

# For each fold, train and test on Linear Regression algorithm
resids <- c(1:nrow(crime_data))
preds <- c(1:nrow(crime_data))
for (i in 1:5) {
    print(glue("Iteration #{i}"))
    test_indices <- which(folds==i, arr.ind=TRUE)
    test_data <- crime_data[test_indices, ]
    train_data <- crime_data[-test_indices, ]

    temp_fit <- lm(formula=Crime ~ M + Ed + Po1 + U2 + Ineq + Prob + Time, data=train_data)
    print(summary(temp_fit))
    
    # Perform prediction step on test data since model was fit using the training data
    preds[test_indices] <- predict(temp_fit, test_data)    
    resids[test_indices] <- test_data$Crime - preds[test_indices]
}

overall_rmse <- round(rmse(resids), 3)
overall_mae <- round(mae(resids), 3)

print(glue("RMSE: {overall_rmse}"))
print(glue("MAE: {overall_mae}"))

Iteration #1

Call:
lm(formula = Crime ~ M + Ed + Po1 + U2 + Ineq + Prob + Time, 
    data = train_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-463.41 -113.86    0.58   68.57  592.99 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -4568.594   1241.738  -3.679 0.000949 ***
M             106.720     38.893   2.744 0.010304 *  
Ed            176.508     59.748   2.954 0.006163 ** 
Po1           109.048     16.902   6.452 4.65e-07 ***
U2             81.040     48.031   1.687 0.102287    
Ineq           58.293     17.056   3.418 0.001891 ** 
Prob        -4109.257   2127.567  -1.931 0.063255 .  
Time           -1.312      6.202  -0.212 0.833895    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 215.3 on 29 degrees of freedom
Multiple R-squared:  0.6924,	Adjusted R-squared:  0.6182 
F-statistic: 9.327 on 7 and 29 DF,  p-value: 5.151e-06

Iteration #2

Call:
lm(formula = Crime ~ M + Ed + Po1 + U2 + I

**The plot is in the appendix.**

In [129]:
xyplot(resids ~ preds,
  xlab = "Fitted Values",
  ylab = "Residuals",
  main = "Residual Diagnostic Plot",
  panel = function(x, y, ...)
  {
    panel.grid(h = -1, v = -1)
    panel.abline(h = 0)
    panel.xyplot(x, y, ...)
  }
)

The second model with the subset of predictors worked the better than the model built on all the predictors. RMSE reduced from 289.558 to 227.23 and MAE reducted from 221.229 to 169.714.

So, now the final model using all available data will be built using the formula below: 

Crime ~ M + Ed + Po1 + U2 + Ineq + Prob + Time


In [15]:
final_model <- lm(formula=Crime ~ M + Ed + Po1 + U2 + Ineq + Prob + Time, data=crime_data)

sample_prediction <- round(predict(final_model, sample_data), 3)

print(glue("Prediction for the crime rate for the sample data is {sample_prediction}"))
print("Final Model")
print(summary(final_model))

Prediction for the crime rate for the sample data is 1285.283
[1] "Final Model"

Call:
lm(formula = Crime ~ M + Ed + Po1 + U2 + Ineq + Prob + Time, 
    data = crime_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-480.89  -89.12   -6.63  140.27  576.79 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -4911.094    960.729  -5.112 8.79e-06 ***
M             106.659     33.877   3.148 0.003144 ** 
Ed            189.408     48.288   3.922 0.000345 ***
Po1           115.704     13.993   8.269 4.16e-10 ***
U2             88.720     41.364   2.145 0.038249 *  
Ineq           67.728     14.083   4.809 2.28e-05 ***
Prob        -4249.756   1880.672  -2.260 0.029502 *  
Time           -2.310      5.538  -0.417 0.678810    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 202.8 on 39 degrees of freedom
Multiple R-squared:  0.7669,	Adjusted R-squared:  0.7251 
F-statistic: 18.33 on 7 and 39 DF,  p-value: 1.553e